In [1]:
import sys
import os
import pandas as pd
import numpy as np
from Bio import SeqIO
import re

In [2]:
dataset = 'RB2022'

print('Loading raw data for', dataset, '...')
data = pd.read_csv('/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/1-s2.0-S1550413122003023-mmc3.csv', header=0)
print('Raw data loaded.')
data

Loading raw data for RB2022 ...


/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_1008/1375056354.py:4: DtypeWarning: Columns (19,37,38,39,40,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/1-s2.0-S1550413122003023-mmc3.csv', header=0)


Raw data loaded.


,Site,Uniprot_ID,Protein name,Gene name,Gene name and site,poiion,Phospho (STY) Probabilities,Sequence window,Log(PostEndurance/PreEndurance),Log(RecoveryEndurance/PreEndurance),...,Subject7_RecoveryStrength,Subject8_PreEndurance,Subject8_PostEndurance,Subject8_RecoveryEndurance,Subject8_PreSprint,Subject8_PostSprint,Subject8_RecoverySprint,Subject8_PreStrength,Subject8_PostStrength,Subject8_RecoveryStrength
0,AAK1;S21;DSRREQGGSGLGSGSSGGGGSTSGLGSGYIG,Q2M2I8,AP2-associated protein kinase 1,AAK1,AAK1;S21,21.0,EQGGSGLGSGS(0.097)S(0.903)GGGGSTSGLGSGYIGR,DSRREQGGSGLGSGSSGGGGSTSGLGSGYIG,0.000242,0.128806,...,0.034006,-0.117119,0.185287,0.704742,-0.051076,0.149990,0.305059,0.038202,0.142266,0.812406
1,AAK1;S624;PAVQGQKVGSLTPPSSPKTQRAGHRRILSDV,Q2M2I8,AP2-associated protein kinase 1,AAK1,AAK1;S624,624.0,VGS(0.007)LT(0.993)PPS(0.069)S(0.931)PK,PAVQGQKVGSLTPPSSPKTQRAGHRRILSDV,-0.375687,-0.004770,...,0.371917,0.461684,0.175775,0.923458,0.442444,0.233604,0.411851,0.233630,0.645927,0.048658
2,AAK1;S846;SLIPGLEPPVPQRLPSQTESVTSNRTDSLTG,Q2M2I8,AP2-associated protein kinase 1,AAK1,AAK1;S846,846.0,LPS(0.999)QT(0.001)ESVTSNR,SLIPGLEPPVPQRLPSQTESVTSNRTDSLTG,0.004786,-0.001989,...,NaN,-0.061904,-0.237457,0.344187,-0.495582,-0.610258,-0.088353,-0.356095,0.279244,-0.019146
3,AAK1;T605;EPAIQAPVRQQPKVQTTPPPAVQGQKVGSLT,Q2M2I8,AP2-associated protein kinase 1,AAK1,AAK1;T605,605.0,VQT(0.923)T(0.077)PPPAVQGQK,EPAIQAPVRQQPKVQTTPPPAVQGQKVGSLT,0.010659,0.036770,...,0.155386,0.159399,0.186964,0.463407,0.325241,0.222430,0.293541,0.121111,0.236358,0.148079
4,AAK1;T606;PAIQAPVRQQPKVQTTPPPAVQGQKVGSLTP,Q2M2I8,AP2-associated protein kinase 1,AAK1,AAK1;T606,606.0,VQT(0.009)T(0.991)PPPAVQGQK,PAIQAPVRQQPKVQTTPPPAVQGQKVGSLTP,-0.111468,0.015810,...,0.217235,0.216737,0.155634,0.729217,0.117497,0.038930,0.142377,0.047696,0.431236,0.002678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9491,ZYX;S249;FSPVTPKFTPVASKFSPGAPGGSGSQPNQKL,H0Y2Y8,Zyxin,ZYX,ZYX;S249,249.0,FS(1)PGAPGGSGSQPNQK,FSPVTPKFTPVASKFSPGAPGGSGSQPNQKL,-0.049213,-0.062891,...,0.512867,1.046673,0.192727,0.940291,-0.180819,-0.371681,0.906592,-0.277645,1.094450,-0.106073
9492,ZYX;S276;NQKLGHPEALSAGTGSPQPPSFTYAQQREKP,H0Y2Y8,Zyxin,ZYX,ZYX;S276,276.0,LGHPEALSAGT(0.002)GS(0.998)PQPPSFTYAQQR,NQKLGHPEALSAGTGSPQPPSFTYAQQREKP,-0.148761,-0.075337,...,0.143604,0.249560,0.030681,0.357808,0.186773,-0.044353,0.522462,0.053053,0.096875,0.110059
9493,ZYX;S312;QHPVPPPAQNQNQVRSPGAPGPLTLKEVEEL,H0Y2Y8,Zyxin,ZYX,ZYX;S312,312.0,S(1)PGAPGPLTLK,QHPVPPPAQNQNQVRSPGAPGPLTLKEVEEL,0.896424,-0.151463,...,0.161892,0.299340,0.483839,-0.060890,-0.604915,-0.617645,0.720643,-0.717400,0.417126,-0.065795
9494,ZZEF1;S2443;LDERGDREEEVERPVSSPGDPEQKKLDPLEG,O43149,Zinc finger ZZ-type and EF-hand domain-contain...,ZZEF1,ZZEF1;S2443,2443.0,GDREEEVERPVS(0.85)S(0.15)PGDPEQK,LDERGDREEEVERPVSSPGDPEQKKLDPLEG,0.120556,0.008702,...,0.515577,0.246374,0.263431,0.360483,-0.086777,-0.085286,0.120698,0.072250,0.167757,0.553560


In [3]:
data.columns = data.columns.str.strip()

In [4]:
# filter data
data['Sequence window'] = data['Sequence window'].str.replace('_', '')

In [5]:
def match_seq_to_genename(dataset, seq_column):
    '''
    Maps amino acid sequences to gene names using the loaded fasta file.
    
    args:
    =====
    dataset: <pd.Dataframe> with a column of amino acid sequences
    seq_column: <str> column name containing amino acid sequences
    
    out:
    ====
    dataset: <pd.Dataframe> with an additional column containing gene names
    '''    
    
    fasta_sequence = list(SeqIO.parse(f"/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/raw_data/UP000005640_9606.fasta", "fasta"))
    
    gene_dict = {}
    
    # iterate over rows in seq_column
    for i in dataset[seq_column]:
        print(i)
        i_str = str(i)
        for seq_record in fasta_sequence:
            matches = re.findall(i_str, str(seq_record.seq))
            if matches:
                print(f"Match found for sequence: {seq_record}")
                gene_name_match = re.search(r"GN=(\w+)", seq_record.description)
                print('Gene name match:', gene_name_match)
                if gene_name_match:
                    gene_name = gene_name_match.group(1)
                    gene_dict[i] = gene_name
                    print(f"Match found: {i_str} -> {gene_name}")
                else:
                    print(f"No gene name match found in description for sequence: {i_str}")

    # map sequences to gene names           
    dataset['GeneName'] = dataset[seq_column].map(gene_dict) 
    print('Amino acid sequences matched to gene names.')

    return dataset

In [6]:
data = match_seq_to_genename(data, 'Sequence window')

DSRREQGGSGLGSGSSGGGGSTSGLGSGYIG
Match found for sequence: ID: sp|Q2M2I8|AAK1_HUMAN
Name: sp|Q2M2I8|AAK1_HUMAN
Description: sp|Q2M2I8|AAK1_HUMAN AP2-associated protein kinase 1 OS=Homo sapiens OX=9606 GN=AAK1 PE=1 SV=3
Number of features: 0
Seq('MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQVTVDEVLAE...IDL')
Gene name match: <re.Match object; span=(77, 84), match='GN=AAK1'>
Match found: DSRREQGGSGLGSGSSGGGGSTSGLGSGYIG -> AAK1
PAVQGQKVGSLTPPSSPKTQRAGHRRILSDV
Match found for sequence: ID: sp|Q2M2I8|AAK1_HUMAN
Name: sp|Q2M2I8|AAK1_HUMAN
Description: sp|Q2M2I8|AAK1_HUMAN AP2-associated protein kinase 1 OS=Homo sapiens OX=9606 GN=AAK1 PE=1 SV=3
Number of features: 0
Seq('MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQVTVDEVLAE...IDL')
Gene name match: <re.Match object; span=(77, 84), match='GN=AAK1'>
Match found: PAVQGQKVGSLTPPSSPKTQRAGHRRILSDV -> AAK1
SLIPGLEPPVPQRLPSQTESVTSNRTDSLTG
Match found for sequence: ID: sp|Q2M2I8|AAK1_HUMAN
Name: sp|Q2M2I8|AAK1_HUMAN
Description: sp|Q2M2I8|AAK1_HUMAN 

KeyboardInterrupt: 

In [16]:
data

,Site,Uniprot_ID,Protein name,Gene name,Gene name and site,poiion,Phospho (STY) Probabilities,Sequence window,Log(PostEndurance/PreEndurance),Log(RecoveryEndurance/PreEndurance),...,Subject8_PreEndurance,Subject8_PostEndurance,Subject8_RecoveryEndurance,Subject8_PreSprint,Subject8_PostSprint,Subject8_RecoverySprint,Subject8_PreStrength,Subject8_PostStrength,Subject8_RecoveryStrength,GeneName
0,AAK1;S21;DSRREQGGSGLGSGSSGGGGSTSGLGSGYIG,Q2M2I8,AP2-associated protein kinase 1,AAK1,AAK1;S21,21.0,EQGGSGLGSGS(0.097)S(0.903)GGGGSTSGLGSGYIGR,DSRREQGGSGLGSGSSGGGGSTSGLGSGYIG,0.000242,0.128806,...,-0.117119,0.185287,0.704742,-0.051076,0.149990,0.305059,0.038202,0.142266,0.812406,AAK1
1,AAK1;S624;PAVQGQKVGSLTPPSSPKTQRAGHRRILSDV,Q2M2I8,AP2-associated protein kinase 1,AAK1,AAK1;S624,624.0,VGS(0.007)LT(0.993)PPS(0.069)S(0.931)PK,PAVQGQKVGSLTPPSSPKTQRAGHRRILSDV,-0.375687,-0.004770,...,0.461684,0.175775,0.923458,0.442444,0.233604,0.411851,0.233630,0.645927,0.048658,AAK1
2,AAK1;S846;SLIPGLEPPVPQRLPSQTESVTSNRTDSLTG,Q2M2I8,AP2-associated protein kinase 1,AAK1,AAK1;S846,846.0,LPS(0.999)QT(0.001)ESVTSNR,SLIPGLEPPVPQRLPSQTESVTSNRTDSLTG,0.004786,-0.001989,...,-0.061904,-0.237457,0.344187,-0.495582,-0.610258,-0.088353,-0.356095,0.279244,-0.019146,AAK1
3,AAK1;T605;EPAIQAPVRQQPKVQTTPPPAVQGQKVGSLT,Q2M2I8,AP2-associated protein kinase 1,AAK1,AAK1;T605,605.0,VQT(0.923)T(0.077)PPPAVQGQK,EPAIQAPVRQQPKVQTTPPPAVQGQKVGSLT,0.010659,0.036770,...,0.159399,0.186964,0.463407,0.325241,0.222430,0.293541,0.121111,0.236358,0.148079,AAK1
4,AAK1;T606;PAIQAPVRQQPKVQTTPPPAVQGQKVGSLTP,Q2M2I8,AP2-associated protein kinase 1,AAK1,AAK1;T606,606.0,VQT(0.009)T(0.991)PPPAVQGQK,PAIQAPVRQQPKVQTTPPPAVQGQKVGSLTP,-0.111468,0.015810,...,0.216737,0.155634,0.729217,0.117497,0.038930,0.142377,0.047696,0.431236,0.002678,AAK1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9491,ZYX;S249;FSPVTPKFTPVASKFSPGAPGGSGSQPNQKL,H0Y2Y8,Zyxin,ZYX,ZYX;S249,249.0,FS(1)PGAPGGSGSQPNQK,FSPVTPKFTPVASKFSPGAPGGSGSQPNQKL,-0.049213,-0.062891,...,1.046673,0.192727,0.940291,-0.180819,-0.371681,0.906592,-0.277645,1.094450,-0.106073,ZYX
9492,ZYX;S276;NQKLGHPEALSAGTGSPQPPSFTYAQQREKP,H0Y2Y8,Zyxin,ZYX,ZYX;S276,276.0,LGHPEALSAGT(0.002)GS(0.998)PQPPSFTYAQQR,NQKLGHPEALSAGTGSPQPPSFTYAQQREKP,-0.148761,-0.075337,...,0.249560,0.030681,0.357808,0.186773,-0.044353,0.522462,0.053053,0.096875,0.110059,ZYX
9493,ZYX;S312;QHPVPPPAQNQNQVRSPGAPGPLTLKEVEEL,H0Y2Y8,Zyxin,ZYX,ZYX;S312,312.0,S(1)PGAPGPLTLK,QHPVPPPAQNQNQVRSPGAPGPLTLKEVEEL,0.896424,-0.151463,...,0.299340,0.483839,-0.060890,-0.604915,-0.617645,0.720643,-0.717400,0.417126,-0.065795,ZYX
9494,ZZEF1;S2443;LDERGDREEEVERPVSSPGDPEQKKLDPLEG,O43149,Zinc finger ZZ-type and EF-hand domain-contain...,ZZEF1,ZZEF1;S2443,2443.0,GDREEEVERPVS(0.85)S(0.15)PGDPEQK,LDERGDREEEVERPVSSPGDPEQKKLDPLEG,0.120556,0.008702,...,0.246374,0.263431,0.360483,-0.086777,-0.085286,0.120698,0.072250,0.167757,0.553560,ZZEF1


In [17]:
# Split the 'Site' column by semicolon
site_split = data['Site'].str.split(';', expand=True)

# Extract amino acid (first character of second element)
data['Amino_Acid'] = site_split[1].str[0]

# Extract position (remaining characters of second element)
data['Position'] = site_split[1].str[1:].astype(int)  # Optional: convert to integer


In [18]:
data['Phosphosite'] = data['Amino_Acid'].astype(str) + '(' + data['Position'].astype(str) + ')'

In [ ]:
# Debugging: Check if 'Phosphosite' column is present after creation
if 'Phosphosite' not in data.columns:
    print("Error: 'Phosphosite' column not created!")
else:
    print("Phosphosite column created successfully.")

data

In [19]:
# Keep only 'Phosphosite' and columns
keepcols = ['Phosphosite', 'GeneName'] + [col for col in data.columns if col.startswith('Subject')]

data = data[keepcols]
data

,Phosphosite,GeneName,Subject1_PreEndurance,Subject1_PostEndurance,Subject1_RecoveryEndurance,Subject1_PreSprint,Subject1_PostSprint,Subject1_RecoverySprint,Subject1_PreStrength,Subject1_PostStrength,...,Subject7_RecoveryStrength,Subject8_PreEndurance,Subject8_PostEndurance,Subject8_RecoveryEndurance,Subject8_PreSprint,Subject8_PostSprint,Subject8_RecoverySprint,Subject8_PreStrength,Subject8_PostStrength,Subject8_RecoveryStrength
0,S(21),AAK1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.034006,-0.117119,0.185287,0.704742,-0.051076,0.149990,0.305059,0.038202,0.142266,0.812406
1,S(624),AAK1,0.700291,0.317863,0.278539,0.713965,0.489725,0.179480,0.305539,0.123498,...,0.371917,0.461684,0.175775,0.923458,0.442444,0.233604,0.411851,0.233630,0.645927,0.048658
2,S(846),AAK1,-0.272709,-0.102103,-0.375250,-0.273168,-0.359730,-0.096767,-0.392508,0.760269,...,NaN,-0.061904,-0.237457,0.344187,-0.495582,-0.610258,-0.088353,-0.356095,0.279244,-0.019146
3,T(605),AAK1,0.232161,0.434993,0.111267,0.363504,0.239713,0.208368,0.205257,0.193332,...,0.155386,0.159399,0.186964,0.463407,0.325241,0.222430,0.293541,0.121111,0.236358,0.148079
4,T(606),AAK1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.217235,0.216737,0.155634,0.729217,0.117497,0.038930,0.142377,0.047696,0.431236,0.002678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9491,S(249),ZYX,0.309382,0.621365,0.304274,0.424323,0.389915,0.472371,0.080228,0.409413,...,0.512867,1.046673,0.192727,0.940291,-0.180819,-0.371681,0.906592,-0.277645,1.094450,-0.106073
9492,S(276),ZYX,0.265968,0.150550,0.184385,0.133944,0.138646,0.249223,0.206164,0.122053,...,0.143604,0.249560,0.030681,0.357808,0.186773,-0.044353,0.522462,0.053053,0.096875,0.110059
9493,S(312),ZYX,-0.295221,0.636340,-0.299310,-0.306493,-0.167398,0.091236,-0.382257,0.716817,...,0.161892,0.299340,0.483839,-0.060890,-0.604915,-0.617645,0.720643,-0.717400,0.417126,-0.065795
9494,S(2443),ZZEF1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.515577,0.246374,0.263431,0.360483,-0.086777,-0.085286,0.120698,0.072250,0.167757,0.553560


In [20]:
subject_columns = [col for col in data.columns if col.startswith('Subject')]
data[subject_columns] = data[subject_columns].apply(pd.to_numeric, errors='coerce')

/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_1667/1296071596.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[subject_columns] = data[subject_columns].apply(pd.to_numeric, errors='coerce')


In [21]:
def log2_transform(dataset):
    '''
    Log2 transform a dataset.
    
    args:
    =====
    dataset: <pd.DataFrame>
    
    out:
    ====
    dataset: <pd.DataFrame> with log2 transformed values
    '''
    cols_to_transform = [col for col in dataset.columns if col.startswith('Subject')]
    dataset[cols_to_transform] = np.log2(dataset[cols_to_transform])
    print('Data has been log2 transformed.')
    return dataset

In [22]:
# Apply the log2 transformation
data = log2_transform(data)
print(f"DataFrame after log2 transformation:\n{data}")

/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/.venv/lib/python3.13/site-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: invalid value encountered in log2
  result = func(self.values, **kwargs)
/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_1667/605153469.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[cols_to_transform] = np.log2(dataset[cols_to_transform])


Data has been log2 transformed.
DataFrame after log2 transformation:
     Phosphosite GeneName  Subject1_PreEndurance  Subject1_PostEndurance  \
0          S(21)     AAK1                    NaN                     NaN   
1         S(624)     AAK1              -0.513974               -1.653524   
2         S(846)     AAK1                    NaN                     NaN   
3         T(605)     AAK1              -2.106805               -1.200934   
4         T(606)     AAK1                    NaN                     NaN   
...          ...      ...                    ...                     ...   
9491      S(249)      ZYX              -1.692538               -0.686488   
9492      S(276)      ZYX              -1.910677               -2.731689   
9493      S(312)      ZYX                    NaN               -0.652130   
9494     S(2443)    ZZEF1                    NaN                     NaN   
9495     S(2444)    ZZEF1                    NaN                     NaN   

      Subject1_Rec

In [23]:
def create_phos_ID(dataset):
    '''
    Concatenates GeneName and Phosphosite columns.
    
    args:
    =====
    dataset: <pd.Dataframe> with columns 'GeneName' and 'Phosphosite'
    
    out:
    ====
    dataset: <pd.Dataframe> with 'phosphosite_ID' column and 'GeneName' + 'Phosphosite' columns dropped
    '''
    dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)
    dataset = dataset.drop(columns=['Phosphosite', 'GeneName'])
    print('Phosphosite IDs created.')
    return dataset

data = create_phos_ID(data) # call function to create phosphosite_ID column

print('Phosphosite IDs created.')

Phosphosite IDs created.
Phosphosite IDs created.


/var/folders/1v/y_z33m9d25d2tszlfrvnp2300000gn/T/ipykernel_1667/3553972964.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[:, 'phosphosite_ID'] = dataset['GeneName'].astype(str) + '_' + dataset['Phosphosite'].astype(str)


In [24]:
def clean_phosID_col(data):
    data = data[~data.phosphosite_ID.str.contains('nan', case = False)]
    data = data[~data.phosphosite_ID.str.contains(';', case = False)] # remove rows containing ';' in phosphosite_ID column
    data = data[~data.phosphosite_ID.str.contains('-', case = False)] # remove rows containing '-' in phosphosite_ID column
    
    # check whether there are any phosphosites with multiple measurements
    data_grouped = data.groupby(by = 'phosphosite_ID')
    if len(data) != len(data_grouped):
        data = data_grouped.mean()
        data.reset_index(inplace=True) # reset index
        print('Phosphosites with multiple measurements have been averaged')
    else:
        print('There are no phosphosites with multiple measurements')
        
    print(data)
        
    data = data.replace([np.inf, -np.inf], np.nan)
        
    if data.columns[0] != 'phosphosite_ID':
        phosphosite_ID = data.pop('phosphosite_ID')
        data.insert(0, 'phosphosite_ID', phosphosite_ID)
    return data



In [25]:
data = clean_phosID_col(data)
data = data[['phosphosite_ID'] + subject_columns]
print("After cleaning phosphosite_ID column:")
data

Phosphosites with multiple measurements have been averaged
     phosphosite_ID  Subject1_PreEndurance  Subject1_PostEndurance  \
0        AAK1_S(21)                    NaN                     NaN   
1       AAK1_S(624)              -0.513974               -1.653524   
2       AAK1_S(846)                    NaN                     NaN   
3       AAK1_T(605)              -2.106805               -1.200934   
4       AAK1_T(606)                    NaN                     NaN   
...             ...                    ...                     ...   
8662     ZYX_S(249)              -1.692538               -0.686488   
8663     ZYX_S(276)              -1.910677               -2.731689   
8664     ZYX_S(312)                    NaN               -0.652130   
8665  ZZEF1_S(2443)                    NaN                     NaN   
8666  ZZEF1_S(2444)                    NaN                     NaN   

      Subject1_RecoveryEndurance  Subject1_PreSprint  Subject1_PostSprint  \
0                      

,phosphosite_ID,Subject1_PreEndurance,Subject1_PostEndurance,Subject1_RecoveryEndurance,Subject1_PreSprint,Subject1_PostSprint,Subject1_RecoverySprint,Subject1_PreStrength,Subject1_PostStrength,Subject1_RecoveryStrength,...,Subject7_RecoveryStrength,Subject8_PreEndurance,Subject8_PostEndurance,Subject8_RecoveryEndurance,Subject8_PreSprint,Subject8_PostSprint,Subject8_RecoverySprint,Subject8_PreStrength,Subject8_PostStrength,Subject8_RecoveryStrength
0,AAK1_S(21),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-4.878053,NaN,-2.432163,-0.504834,NaN,-2.737063,-1.712839,-4.710198,-2.813337,-0.299727
1,AAK1_S(624),-0.513974,-1.653524,-1.844049,-0.486075,-1.029955,-2.478103,-1.710573,-3.017436,-1.095017,...,-1.426948,-1.115022,-2.508195,-0.114881,-1.176434,-2.097864,-1.279807,-2.097703,-0.630557,-4.361191
2,AAK1_S(846),NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.395418,NaN,...,NaN,NaN,NaN,-1.538736,NaN,NaN,NaN,NaN,-1.840400,NaN
3,AAK1_T(605),-2.106805,-1.200934,-3.167899,-1.459957,-2.060623,-2.262794,-2.284497,-2.370844,-2.574037,...,-2.686069,-2.649288,-2.419167,-1.109647,-1.620420,-2.168576,-1.768368,-3.045601,-2.080957,-2.755565
4,AAK1_T(606),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.202674,-2.205985,-2.683771,-0.455580,-3.089306,-4.682958,-2.812208,-4.389995,-1.213451,-8.544424
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8662,ZYX_S(249),-1.692538,-0.686488,-1.716559,-1.236765,-1.358767,-1.082008,-3.639759,-1.288373,-2.100078,...,-0.963343,0.065811,-2.375371,-0.088821,NaN,NaN,-0.141474,NaN,0.130206,NaN
8663,ZYX_S(276),-1.910677,-2.731689,-2.439203,-2.900296,-2.850519,-2.004490,-2.278135,-3.034415,-3.158635,...,-2.799834,-2.002542,-5.026531,-1.482741,-2.420643,NaN,-0.936603,-4.236413,-3.367732,-3.183648
8664,ZYX_S(312),NaN,-0.652130,NaN,NaN,NaN,-3.454251,NaN,-0.480324,NaN,...,-2.626899,-1.740141,-1.047402,NaN,NaN,NaN,-0.472644,NaN,-1.261445,NaN
8665,ZZEF1_S(2443),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.955740,-2.021076,-1.924501,-1.471996,NaN,NaN,-3.050528,-3.790862,-2.575555,-0.853187


In [26]:
data.to_csv(f'/Users/maryamkoddus/Documents/maryam-ko-QMUL-MSc-Project/01_input_data/PreprocessedDatasets/RB2022.csv', index=False)


print(dataset, 'has been saved to CSV successfully!', data)                       

RB2022 has been saved to CSV successfully!      phosphosite_ID  Subject1_PreEndurance  Subject1_PostEndurance  \
0        AAK1_S(21)                    NaN                     NaN   
1       AAK1_S(624)              -0.513974               -1.653524   
2       AAK1_S(846)                    NaN                     NaN   
3       AAK1_T(605)              -2.106805               -1.200934   
4       AAK1_T(606)                    NaN                     NaN   
...             ...                    ...                     ...   
8662     ZYX_S(249)              -1.692538               -0.686488   
8663     ZYX_S(276)              -1.910677               -2.731689   
8664     ZYX_S(312)                    NaN               -0.652130   
8665  ZZEF1_S(2443)                    NaN                     NaN   
8666  ZZEF1_S(2444)                    NaN                     NaN   

      Subject1_RecoveryEndurance  Subject1_PreSprint  Subject1_PostSprint  \
0                            NaN       